In [76]:
from solve import first_fit_solve, first_fit_with_rotate, backtrack_solve
from classes import Node
from input_output import convert_to_solution_file
from time import perf_counter

solve = backtrack_solve
output_path = "./checker/checker/instances_checker" #"solutions"


def run_all(dataset="A", log_file: dict = []):
    for i in range(1, 21):
        Node.reset_id_counter()
        param = f"{dataset}{i}"
        run_one(param, log_file)


def run_one(param="A1", log_file:dict = {}):
    start_time = perf_counter()

    Node.reset_id_counter()
    solution_trees = solve(param, 2, True)

    convert_to_solution_file(solution_trees, param, output_path)
    end_time = perf_counter()
    runtime = end_time - start_time
    print(f"\truntime: {runtime:.2f} seconds")
    log_file[param]["runtime"] = runtime

In [77]:
import os 

def go_to_checker():
    os.chdir("d:\\Suli\\MsC\\3.semester\\Opkut projekt\\Glass-cutting\\checker\\checker")

def go_to_main_py():
    os.chdir("d:\\Suli\\MsC\\3.semester\\Opkut projekt\\Glass-cutting")

In [ ]:
import subprocess
import threading
import time

def get_checker_logs_for_dataset(id:str = 'A', log_file:dict = {}):
    "Checks solution for all instances in a dataset"

    instance_dct = {'A': 20, 'B': 15, 'X':15}
    instance_num = instance_dct[id]

    for i in range(instance_num + 1 ):
        get_checker_logs_for_instance(f'{id}{i}', log_file)


def get_checker_logs_for_instance(id:str = 'A1', log_file:dict = {}):
    """
    Runs the checker function for the given id dataset. 
    Returns the list of logs
    """
    log = []

    # Start the main.exe process with buffering and stdout/stderr handling
    process = subprocess.Popen(
        ["main.exe"],  # Replace with your actual path to main.exe
        stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True,
        universal_newlines=True, bufsize=1, shell=True
    )

    # Function to handle reading stdout asynchronously
    def read_stdout(process, log_file):
        while True:
            output = process.stdout.readline()
            if output:
                print("STDOUT:", output.strip())
                log_file.append(output.strip())
            else:
                break  # Process finished or no more output

    # Function to handle reading stderr asynchronously
    def read_stderr(process, log_file):
        while True:
            error = process.stderr.readline()
            if error:
                print("STDERR:", error.strip())
                log_file.append(error.strip())
            else:
                break  # Process finished or no more errors

    # Start threads to read stdout and stderr without blocking
    stdout_thread = threading.Thread(target=read_stdout, args=(process, log))
    stderr_thread = threading.Thread(target=read_stderr, args=(process, log))

    stdout_thread.start()
    stderr_thread.start()

    # Step 1: Send input to the process (e.g., A1 dataset)
    process.stdin.write(f"{id}\n")
    process.stdin.flush()

    # Wait a bit to simulate waiting for the program to process the input
    time.sleep(1)

    # Step 2: Get the validity of solution (all condition)
    process.stdin.write("5\n")
    process.stdin.flush()

    # Wait for output
    time.sleep(1)

    # Get the used area and Objective Function value
    process.stdin.write("6\n")
    process.stdin.flush()

    # Wait for output
    time.sleep(1)

    # Close the program
    process.stdin.write("0\n")
    process.stdin.flush()

    # Wait for both threads to finish (the process will terminate when no more output is available)
    stdout_thread.join()
    stderr_thread.join()

    # Close the process properly
    process.stdin.close()
    process.stdout.close()
    process.stderr.close()
    process.wait()

    log_file[id]["log_file"] = log

    


In [98]:
import pandas as pd

def get_useful_data(log_dict: dict, key: str) -> dict:
    log_file = log_dict[key]["log_file"]
    res = {"id": key}
    for line in log_file:
        if "SOLUTION" in line:
            res["Success"] = ("SUCCESSFULLY" in line)
        if "Total" in line:
            line = line.split('|')
            res["Used_area"] = float(line[2].strip("%\t").strip())
        if 'OF' in line:
            line = line.split('=')[1].strip('-').strip()
            res["OF"] = int(line)
    res["runtime"] = log_dict[key].get("runtime",0)
    return res

def logs_to_csv(log_dict, id):
    data = []
    for key in log_dict.keys():
        res = get_useful_data(log_dict, key)
        data.append(res)
    
    df = pd.DataFrame(data)
    # save to CSV
    csv_file_path = os.path.join(f"{id}_solution_summary.csv")

    df.to_csv(csv_file_path, sep=";", index=False)

In [ ]:
# I want to run all test cases for a dataset 
# I want to run the checker on them 
# I want to clear the checker's solution 
# Put them into a csv and run analytics 
from collections import defaultdict

def run_solver_and_check_solution():
    log_file = defaultdict(dict)

    go_to_main_py()
    run_all(log_file = log_file)
    go_to_checker()

    get_checker_logs_for_dataset('A', log_file)
    logs_to_csv(log_file, id = "A")

def run_checker():
    log_file = defaultdict(dict)
    go_to_checker()
    get_checker_logs_for_dataset('A', log_file)
    logs_to_csv(log_file, id = "A")


In [ ]:
log_file = defaultdict(dict)

# go_to_main_py()
# run_all(log_file = log_file)
go_to_checker()

get_checker_logs_for_dataset('A', log_file)
logs_to_csv(log_file, id = "A")

STDOUT: Please enter your USED BATCH file Index:
STDOUT: ------------------- Optimization Checker -------------------
STDOUT: 
STDOUT: 
STDOUT: ------------------- Version - 23/05/2018 -------------------
STDOUT: 
STDOUT: 
STDOUT: ----------- Start files parsing -----------
STDOUT: 
STDOUT: --- Parsing Optimization Parameters file ---
STDOUT: Optimization Parameters file path: instances_checker/global_param.csv
STDOUT: nPlates: 100
STDOUT: widthPlates: 6000
STDOUT: heightPlates: 3210
STDOUT: min1Cut: 100
STDOUT: max1Cut: 3500
STDOUT: min2Cut: 100
STDOUT: minWaste: 20
STDOUT: --- Optimization Parameters file parsed successfully ---
STDOUT: 
STDOUT: --- Parsing Batch file ---
STDOUT: Batch file path: instances_checker/A0_batch.csv
STDOUT: Items: 3
STDOUT: Stacks: 1
STDOUT: --- Batch file parsed successfully ---
STDOUT: 
STDOUT: --- Parsing Solution file ---
STDOUT: Solution file path: instances_checker/A0_solution.csv
STDOUT: Plates: 1
STDOUT: Node 9: is a residual
STDOUT: Nodes: 10
STDO